In [39]:
import pandas as pd

In [40]:
df=pd.read_csv('/content/drive/MyDrive/Datasets/fake_news/train_news.csv',index_col=False) # Reading the data from drive

In [41]:
df.drop(df.columns[0],axis=1,inplace=True) # Dropping extra column of axis

In [42]:
df

,id,headline,written_by,news,label
0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo...",0
1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...,0
2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ...",0
3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g...",0
4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,1
...,...,...,...,...,...
20795,5671,NaN,NeverSurrender,"No, you'll be a dog licking of the vomit of yo...",1
20796,14831,Albert Pike and the European Migrant Crisis,Rixon Stewart,"By Rixon Stewart on November 5, 2016 Rixon Ste...",1
20797,18142,Dakota Access Caught Infiltrating Protests to ...,Eddy Lavine,posted by Eddie You know the Dakota Access Pip...,1
20798,12139,How to Stretch the Summer Solstice - The New Y...,Alison S. Cohn,"It’s officially summer, and the Society Boutiq...",0


In [43]:
X=df.drop('label',axis=1) # Droping output feature

In [44]:
X.head()

,id,headline,written_by,news
0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo..."
1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...
2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ..."
3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g..."
4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...


In [45]:
y=df['label'] # Assigning output to y

In [46]:
y.head()

0    0
1    0
2    0
3    0
4    1
Name: label, dtype: int64

In [47]:
df.shape

(20800, 5)

In [48]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [49]:
df=df.dropna() # Droping null values present

In [50]:
df.shape

(18285, 5)

In [51]:
news=df.copy()

In [52]:
news.reset_index(inplace=True)

In [53]:
news.head(10)

,index,id,headline,written_by,news,label
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo...",0
1,1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...,0
2,2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ...",0
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g...",0
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,1
5,6,5134,Rapid Evolution Saved This Fish From Pollution...,JoAnna Klein,The State of New Jersey says you can’t eat the...,0
6,7,1504,Alabama Prison Officials Retaliate Against Pri...,Brian Sonenstein,Advocates say prison officials at the Kilby Co...,1
7,9,4203,Can We Live in a Constant State of Love?,Gillian,Leave a reply \nToni Emerson – When we fall in...,1
8,10,11136,Is Hillary a Closet Drunk… New WikiLeak Email ...,V Saxena,BREAKING: Clinton Fixer Reveals Sick Smear Hil...,1
9,11,2150,BLOOMBERG-BACKED PENNSYLVANIA ATTORNEY GENERAL...,Iron Sheik,Home › POLITICS | US NEWS › BLOOMBERG-BACKED P...,1


In [54]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Taking input as headline


In [55]:

import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(news)):
    headline = re.sub('[^a-zA-Z]', ' ', news['headline'][i]) # Filtering all the headlines by removing numbers and symbols
    headline = headline.lower()
    headline = headline.split()
    
    headline = [ps.stem(word) for word in headline if not word in stopwords.words('english')] # making base form of the words
    headline = ' '.join(headline)
    corpus.append(headline)

In [56]:
corpus[2:5]

['cotton hous walk plank vote bill cannot pass senat breitbart',
 'paul lepag besieg main governor send conflict signal intent new york time',
 'digit trump win']

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,ngram_range=(1,3))
X = cv.fit_transform(corpus).toarray()

In [58]:
X.shape

(18285, 5000)

In [59]:
y=news['label']

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0) # Dividing data in train test

In [61]:

cv.get_feature_names()[:20]

['abandon',
 'abc',
 'abc news',
 'abduct',
 'abe',
 'abedin',
 'abl',
 'abort',
 'abroad',
 'absolut',
 'abstain',
 'absurd',
 'abus',
 'abus new',
 'abus new york',
 'academi',
 'accept',
 'access',
 'access pipelin',
 'access pipelin protest']

In [62]:

cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [63]:

count_df = pd.DataFrame(X_train, columns=cv.get_feature_names())

In [64]:
count_df.head()

,abandon,abc,abc news,abduct,abe,abedin,abl,abort,abroad,absolut,abstain,absurd,abus,abus new,abus new york,academi,accept,access,access pipelin,access pipelin protest,accid,accident,accord,account,accus,accus trump,achiev,acknowledg,acknowledg emf,acknowledg emf damag,aclu,acquit,acquitt,acr,across,act,act like,act new,act new york,action,...,yahoo,yale,ye,year,year ago,year breitbart,year eve,year later,year new,year new york,year old,year old girl,year sinc,yemen,yet,yet anoth,yiannopoulo,yield,yo,york,york citi,york new,york new york,york state,york time,yorker,young,youth,youtub,zealand,zero,zika,zika viru,zionist,zone,zone new,zone new york,zoo,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [65]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()

In [66]:
from sklearn import metrics
import numpy as np
import itertools

In [67]:
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred)
cm

accuracy:   0.902


array([[3039,  351],
       [ 241, 2404]])

In [68]:
from sklearn.linear_model import PassiveAggressiveClassifier
linear_clf = PassiveAggressiveClassifier(max_iter=50)

In [69]:
linear_clf.fit(X_train, y_train)
pred = linear_clf.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred)
cm

accuracy:   0.920


array([[3131,  259],
       [ 221, 2424]])

In [70]:
classifier=MultinomialNB(alpha=0.1)

In [71]:
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    y_pred=sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test, y_pred)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score))

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha: 0.0, Score : 0.8851698425849213
Alpha: 0.1, Score : 0.8984258492129246
Alpha: 0.2, Score : 0.9000828500414251
Alpha: 0.30000000000000004, Score : 0.9005799502899752
Alpha: 0.4, Score : 0.9012427506213753
Alpha: 0.5, Score : 0.9009113504556753
Alpha: 0.6000000000000001, Score : 0.9010770505385253
Alpha: 0.7000000000000001, Score : 0.9019055509527755
Alpha: 0.8, Score : 0.9017398508699255
Alpha: 0.9, Score : 0.9015741507870754


In [72]:
feature_names = cv.get_feature_names()

In [73]:
classifier.coef_[0]

array([ -9.32421836,  -8.7536735 , -10.04334102, ..., -10.79055543,
        -8.49914363, -10.79055543])

In [74]:
sorted(zip(classifier.coef_[0], feature_names), reverse=True)[:20]

[(-3.9675111413474324, 'trump'),
 (-4.262110297382556, 'hillari'),
 (-4.393810972851313, 'clinton'),
 (-4.8536057309316964, 'elect'),
 (-5.200813000454728, 'new'),
 (-5.256287973823915, 'video'),
 (-5.292022012514838, 'us'),
 (-5.319686293987194, 'comment'),
 (-5.352959325324056, 'fbi'),
 (-5.397433057467033, 'war'),
 (-5.459986015348155, 'hillari clinton'),
 (-5.492793769119066, 'email'),
 (-5.526714488715316, 'obama'),
 (-5.592058394467554, 'vote'),
 (-5.642415162652731, 'world'),
 (-5.773275588918455, 'donald'),
 (-5.803150850352285, 'russia'),
 (-5.803150850352285, 'donald trump'),
 (-5.826158000130412, 'say'),
 (-5.826158000130412, 'america')]

In [ ]:
# Passive aggresive classifier predicting better results than other models with good accuracy

Taking input as news


In [76]:
corpus = []
for i in range(0, len(news)):
    news1 = re.sub('[^a-zA-Z]', ' ', news['news'][i])
    news1 = news1.lower()
    news1 = news1.split()
    
    news1 = [ps.stem(word) for word in news1 if not word in stopwords.words('english')]
    news1 = ' '.join(news1)
    corpus.append(news1)

In [78]:
corpus[2:5]

['sunday abc week discuss republican plan repeal replac obamacar sen tom cotton warn gop hous member walk plank vote bill cannot pass senat face consequ vote ad bill probabl fix go take lot carpentri framework hill follow pam key twitter pamkeynen',
 'augusta beleagu republican governor main seem tuesday back hint might quit midst controversi latest tirad obscen threaten racial charg remark republican lawmak hous ralli quash possibl formal rebuk wors errat behavior gov paul r lepag whose tenur often seem lurch one crisi anoth began day suggest radio interview tuesday morn consid step outrag gener profan voic mail appar endors racial profil drew admonish even member parti realli one thing one parti behind mr lepag said radio interview time uncharacterist anoth thing parti behind earli afternoon twitter messag mr lepag seem regain spirit defianc regard rumor resign paraphras mark twain report polit demis greatli exagger messag read mr lepag spend much day town baileyvil mile near canadia

In [79]:
cv = CountVectorizer(max_features=5000,ngram_range=(1,3))
X = cv.fit_transform(corpus).toarray()

In [80]:
X.shape

(18285, 5000)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [82]:
cv.get_feature_names()[:20]

['aaron',
 'abandon',
 'abc',
 'abe',
 'abedin',
 'abil',
 'abl',
 'abort',
 'abroad',
 'absenc',
 'absolut',
 'absorb',
 'absurd',
 'abu',
 'abus',
 'academ',
 'academi',
 'acceler',
 'accept',
 'access']

In [83]:
count_df = pd.DataFrame(X_train, columns=cv.get_feature_names())

In [84]:
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred)
cm

accuracy:   0.898


array([[3122,  268],
       [ 349, 2296]])

In [85]:
linear_clf.fit(X_train, y_train)
pred = linear_clf.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred)
cm

accuracy:   0.950


array([[3230,  160],
       [ 141, 2504]])

In [86]:
classifier=MultinomialNB(alpha=0.1)

In [87]:
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    y_pred=sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test, y_pred)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score))

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha: 0.0, Score : 0.8992543496271748
Alpha: 0.1, Score : 0.8977630488815245
Alpha: 0.2, Score : 0.8975973487986744
Alpha: 0.30000000000000004, Score : 0.8975973487986744
Alpha: 0.4, Score : 0.8975973487986744
Alpha: 0.5, Score : 0.8975973487986744
Alpha: 0.6000000000000001, Score : 0.8975973487986744
Alpha: 0.7000000000000001, Score : 0.8975973487986744
Alpha: 0.8, Score : 0.8975973487986744
Alpha: 0.9, Score : 0.8977630488815245


In [88]:
feature_names = cv.get_feature_names()

In [89]:
linear_clf.coef_[0]

array([-0.00645714, -0.04301443,  0.00921858, ..., -0.06300474,
        0.12124964,  0.04258483])

In [90]:
sorted(zip(linear_clf.coef_[0], feature_names), reverse=True)[:20]

[(0.7989926682571206, 'anti'),
 (0.7923446685611679, 'octob'),
 (0.534209595720022, 'non'),
 (0.5160306662977858, 'co'),
 (0.4878448431598045, 'novemb'),
 (0.4636199616156356, 'self'),
 (0.45715083255720246, 'year old'),
 (0.4453181678301757, 'click'),
 (0.3891130337204917, 'presid elect'),
 (0.3877924067352908, 'howev'),
 (0.38476318572983764, 'hillari'),
 (0.3467425827564658, 'via'),
 (0.34669869394730896, 'sourc'),
 (0.3462326644326611, 'print'),
 (0.310253874199155, 'al'),
 (0.2920621318631994, 'twitter com'),
 (0.2877052728626955, 'jew'),
 (0.2875830249250041, 'com'),
 (0.2801297724523949, 'giuliani'),
 (0.268497331865988, 'share')]

It can be observed that taking input as news giving good accuracy score and predictions are also better when confusion matrix is observed. passive aggresive classifier is giving accuracy of 95 % with news than headline used as input which is giving 92%.